In [ ]:
root = "/root/to/repo/scMultiNet_project"
save_root = "root/to/dataset/"
import sys
sys.path.append(root )

In [ ]:
# start with a vocab list
# this related with the gene2vec model
#----> pre-trained part
vocab_loc = root +"/support_data/vocab_gene2vec_16906.pkl"
target_label = 'Ground_truth' # the label to be predicted

In [ ]:
csv_lung =  "/Path/to/raw/cellline_LUNG_dataset.csv" # if the data is in csv format, you need to firstly convert it to h5ad format
# code to convert csv to h5ad

data_path = "/Path/to/raw/cellline_LUNG_dataset.h5ad"

In [ ]:
# check the data
import scanpy as sc
import pandas as pd
import numpy as np

adata = sc.read(data_path)
print(adata)
print(adata.obs["Ground_truth"].unique())
print(adata.obs["Tissue"].unique())
print(adata.obs["CellLine"].unique())
print(adata.obs["stimulus"].unique())

In [ ]:

from scLLM.Dataset.paras import Dataset_para
# define pre-processing by follow original implementation of scBERT

from scLLM.Dataset.paras import Dataset_para
# define pre-processing by follow original implementation of scBERT
dataset_para_cls = Dataset_para(
                            var_idx=None,
                            obs_idx="Ground_truth",
                            vocab_loc="/home/shi/WorkSpace/projects/scLLM_workspace/vocab_16k.json",
                            filter_gene_by_counts=False,
                            filter_cell_by_counts=200,
                            log1p=True,
                            log1p_base=2,

                            #
                            tokenize_name="scBERT",
                            cls_nb=5,
                            data_layer_name="X_log1p",
                            label_key = target_label,#"Ground_truth",#"Ground_truth",

                            test_size=0.2,#0.2, #use all data to inference
                            binarize=None, # not binarize use original label

                            )

dataset_para_reg = Dataset_para(
        vocab_loc="/home/shi/WorkSpace/projects/scLLM_workspace/vocab_16k.json",
        var_idx = None,#"genes.gene_short_name",
        obs_idx="pseudotimes",
        filter_gene_by_counts=False,
        filter_cell_by_counts=200,
        log1p=True,
        log1p_base=2,

        tokenize_name="scBERT",
        cls_nb=1,
        data_layer_name="X_log1p",

        auto_map_str_labels=False,
        label_key = target_label,#"pseudotimes",

        test_size=0.2,#0.2, #use all data to inference
        binarize=None, # not binarize use original label for regression
    )

dataset_para = dataset_para_cls#dataset_para_cls
print(dataset_para)

In [ ]:
# if this is the first time to run, need this block to init translate=True
# init preprocessor
from scLLM.Dataset.Reader import scReader
screader = scReader(dataset_para=dataset_para)
# init vocab
screader.init_vocab()

# load data
screader.load_adata(loc = data_path,translate=True)

## preprocess
screader.preprocess()

In [ ]:
# if not the first run 
# -----> 读取数据集
from scLLM.Dataset.Reader import scReader
data_reader = scReader(dataset_para)
# init vocab from default file loc or from list/dict given as params
data_reader.init_vocab()
#load anndata
data_reader.load_adata(loc = data_path,translate=False)

In [ ]:
target_stimulate = "TGFb"
target_cellline = None #"A549" 'DU145', 'MCF7', 'OVCA420'
map_stimulus = True
map_cancer_type = False

print(f"选择前的数据矩阵形状 {data_reader.adata.shape}")
if map_stimulus:
    stimilus_map = data_reader.adata.obs['stimulus']==target_stimulate
    # filter out cells that are not stimulated by TGFb
    new_adata = data_reader.adata[stimilus_map,:]
    data_reader.adata = new_adata

if map_cancer_type:
    cancer_type_map = data_reader.adata.obs['CellLine']==target_cellline
    # filter out cells that are not stimulated by TGFb
    new_adata = data_reader.adata[cancer_type_map,:]
    data_reader.adata = new_adata

print(f"选择后的数据矩阵形状 {data_reader.adata.shape}")

In [ ]:
label_unique = data_reader.adata.obs[dataset_para.label_key].unique()
print(label_unique)
label_dict = {'3d':0, '7d':1, '1d':2, '8h':3, '0d':4}

In [ ]:
data_reader.adata.obs[dataset_para.label_key]

In [ ]:
trainset,valset,weights = data_reader.postprocess()

# 输出数据集信息
print("trainset size: ",len(trainset))
print("valset size: ",len(valset)) if valset is not None else print("no valset")
print(label_dict)
print("weights: ",weights)

In [ ]:
import dill
# 为trainset 添加其他labels

dataset_from = "/EMT_Cook/"
#target_task = f"/TrVal_dataset_GT_{target_stimulate}_{target_cellline}.pkl"
target_task = f"/TrVal_dataset_{target_label}.pkl"
loc = save_root + dataset_from + target_task
# 保存 trainset 到文件，并关联相应labels
with open(loc,"wb") as f:
    dill.dump([trainset,valset,weights,label_dict],f)